In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# 라이브러리 설치
!pip install -q openai-whisper jiwer ffmpeg-python
!apt install -y ffmpeg
!pip install -U jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 37.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 92.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
# 경로 설정
import os

DATA_DIR = "/content/drive/MyDrive/중간_프로젝트/voice_data/KsponSpeech_0001"
OUT_DIR = "/content/drive/MyDrive/중간_프로젝트/voice_data/wav_output"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# Whisper 모델 임포트
import whisper
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 220MiB/s]


In [ ]:
# 텍스트 전처리 함수 정의
import re

def normalize_text(text: str) -> str:
    text = re.sub(r'^(([A-Za-zㄱ-ㅎ가-힣]+)\s*/\s*)+', '', text)
    text = re.sub(r'\s*([A-Za-zㄱ-ㅎ가-힣]+)\s*/', r' \1', text)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

In [ ]:
# pcm 파일 샘플링
import random

random.seed(1110)

all_pcm_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".pcm")]

sample_size = 10
pcm_files = random.sample(all_pcm_files, sample_size)

total_wer = 0.0
total_cer = 0.0
count = 0

In [ ]:
# 순회
import subprocess
from jiwer import wer, cer

for fname in pcm_files:
    base = fname[:-4]
    pcm_path = os.path.join(DATA_DIR, fname)
    txt_path = os.path.join(DATA_DIR, base + ".txt")

    # pcm → wav
    wav_path = os.path.join(OUT_DIR, base + ".wav")
    subprocess.run([
        "ffmpeg", "-y",
        "-f", "s16le", "-ar", "16000", "-ac", "1",
        "-i", pcm_path,
        wav_path
    ], check=True)

    # 정답 읽기
    with open(txt_path, encoding = "cp949") as f:
        ref_text = f.read().strip()

    # 예측
    result = model.transcribe(wav_path, language="ko")
    pred_text = result["text"].strip()

    # 전처리
    ref_norm = normalize_text(ref_text)
    pred_norm = normalize_text(pred_text)

    # WER/CER
    file_wer = wer(ref_norm, pred_norm)
    file_cer = cer(ref_norm, pred_norm)

    print("\n정답:", ref_norm)
    print("예측:", pred_norm)
    print(f"WER: {file_wer:.3f}, CER: {file_cer:.3f}")

    total_wer += file_wer
    total_cer += file_cer
    count += 1


정답: 지금 개봉했어?
예측: 지금 개봉했어?
WER: 0.000, CER: 0.000

정답: 나 그때 뭐냐 뭐지? 무슨 영화지?
예측: 나 그때 뭐냐 뭐지 무슨 영화지
WER: 0.333, CER: 0.105

정답: 그러면 그 모자는?
예측: 그러면 그 모자는?
WER: 0.000, CER: 0.000

정답: 그것도 그런데 내일로 가면 어디로 가게 이번엔? 경상도 가
예측: 그럼 또 그런데 내일로 가면 어디로 가게 이번엔? 경상도 가...
WER: 0.333, CER: 0.188

정답: 그것도 복합적인 거 아닐까, 혈액 공급이 제대로 안 되니까.
예측: 그것도 복합적인 거 아닐까? 혈액공급이 제대로 안 되니까
WER: 0.444, CER: 0.091

정답: 집순이가 바빴네. 그거 목요일 얘기 아니야?
예측: 집순이가 바빴네 그거 목요일 얘기 아니야?
WER: 0.167, CER: 0.042

정답: 소리 들어감 안 돼.
예측: 그 소리 들어가면 안돼
WER: 1.000, CER: 0.545

정답: 그러다 보니까 엄마랑 자주 안 마주쳤다가 마주치니까 이게 불화가 생길 때마다 어떻게 해결해야 될지 모르겠는 거야.
예측: 그러다 보니까 엄마랑 자주 안 마주쳤다가 마주치니까 이게 불화가 생길 때마다 어떻게 해결이 될지 모르겠는 거야
WER: 0.125, CER: 0.048

정답: 작년에 안 간 거야, 올해는 다 갔어?
예측: 아 작년에 안 간거야? 올해는 다 갔어?
WER: 0.429, CER: 0.190

정답: 가끔가다가 이제 일단 경기 일정 같은 거 보면서 내가 원하는 내가 보고 싶은 팀들 있잖아. 어 그 팀들 하이라이트 경기들 검색해가지고 보기도 하고 그렇지.
예측: 어.. 뭐 가끔 가다가 이제.. 일단 경기 일정 같은 거 보면서 내가 원하는, 내가 보고 싶은 팀들 있잖아? 그 팀들 하이라이트 경기들 검색해가지고 보기도 하고 그렇지
WER: 0.375, CER: 0.163


In [ ]:
# 평균 출력
if count > 0:
    print(f"평균 WER: {total_wer / count:.3f}")
    print(f"평균 CER: {total_cer / count:.3f}")

평균 WER: 0.321
평균 CER: 0.137
